In [1]:
from pyspark import SparkContext
try:
    sc.stop() #停止以前的SparkContext，要不然下面创建工作会失败A
except:
    pass
sc = SparkContext('local', 'movieLens_cluster')

## 提取电影的题材标签

In [2]:
genres = sc.textFile('hdfs://master:9000/movieLens/ml-100k/u.genre')
print(genres.take(5))

['unknown|0', 'Action|1', 'Adventure|2', 'Animation|3', "Children's|4"]


**为电影题材编码**

In [5]:
genre_map = genres.filter(lambda x: len(x) > 0).\
                map(lambda line: line.split('|')).\
                map(lambda x: (x[1], x[0])).collectAsMap()
print('构造出电影题材的编码字典：', genre_map)

构造出电影题材的编码字典： {'0': 'unknown', '14': 'Romance', '9': 'Fantasy', '2': 'Adventure', '17': 'War', '15': 'Sci-Fi', '4': "Children's", '10': 'Film-Noir', '8': 'Drama', '6': 'Crime', '16': 'Thriller', '3': 'Animation', '5': 'Comedy', '18': 'Western', '1': 'Action', '11': 'Horror', '13': 'Mystery', '12': 'Musical', '7': 'Documentary'}


**提取电影的title和genres**

In [8]:
movies = sc.textFile('hdfs://master:9000/movieLens/ml-100k/u.item')
print('电影数据集的第一条数据：', movies.first())
#查看电影的标题
movies_title = movies.map(lambda x: x.split('|')).map(lambda x: x[1])
print('电影标题：', movies_title.take(5))
#查看电影的题材，0表示不属于该题材，1表示属于该题材
movies_genre = movies.map(lambda x: x.split('|')).map(lambda x: x[5:])
print('电影的题材：')
print(movies_genre.take(5))

电影数据集的第一条数据： 1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
电影标题： ['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', 'Get Shorty (1995)', 'Copycat (1995)']
电影的题材：
[['0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0'], ['0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0']]


In [10]:
def getTitleAndGenreName(rdd):
    genres = rdd[5:]
    genres_assigned = zip(genres, range(len(genres)))
    index = [] #存储题材特征值为1的特征索引号
    for genre, idx in genres_assigned:
        if genre == '1':
            index.append(idx)
    index_val = [genre_map[str(i)] for i in index] #根据编码字典找出索引的相应题材名
    index_val_str = ','.join(index_val)
    
    return (int(rdd[0]), rdd[1] + ',' + index_val_str)

titles_and_genres = movies.map(lambda x: x.split('|')).map(lambda x: getTitleAndGenreName(x))
print('前五部电影的标题和相应的题材类型：', titles_and_genres.take(5))

前五部电影的标题和相应的题材类型： [(1, "Toy Story (1995),Animation,Children's,Comedy"), (2, 'GoldenEye (1995),Action,Adventure,Thriller'), (3, 'Four Rooms (1995),Thriller'), (4, 'Get Shorty (1995),Action,Comedy,Drama'), (5, 'Copycat (1995),Crime,Drama,Thriller')]


## 训练推荐模型

### 交替最小二乘法（ALS）
ALS 的核心就是下面这个假设：打分矩阵A是近似低秩的。换句话说，一个m×n的打分矩阵**A**可以用两个小矩阵U(m×k)和V(n×k)的乘积来近似。这样我们就把整个系统的自由度从一下O(mn)降到了O((m+n)×k)  
世上万千事物，人们的喜好各不相同。但描述一个人的喜好经常是在一个抽象的低维空间上进行的，并不需要把其喜欢的事物一一列出。  
举个例子，我喜欢看略带黑色幽默的警匪电影，那么大家根据这个描述就知道我大概会喜欢昆汀的《低俗小说》、《落水狗》和韦家辉的《一个字头的诞生》。这些电影都符合我对自己喜好的描述，也就是说他们在这个抽象的低维空间的投影和我的喜好相似。  
再抽象一些，把人们的喜好和电影的特征都投到这个低维空间，一个人的喜好映射到了一个低维向量，一个电影的特征变成了纬度相同的向量，那么这个人和这个电影的相似度就可以表述成这两个向量之间的内积。  
我们把打分理解成相似度，那么“打分矩阵A(m*n)”就可以由“用户喜好特征矩阵U(m*k)”和“产品特征矩阵V(n*k)”的乘积来近似了。  
具体过程查看[矩阵分解模型（1）：ALS学习算法](https://blog.csdn.net/oucpowerman/article/details/49847979)

现在可以开始训练模型了,所需的其他参数有以下几个。
* rank:对应ALS模型中的因子个数,也就是在低阶近似矩阵中的隐含特征个数。因子个数一般越多越好。但它也会直接影响模型训练和保存时所需的内存开销,尤其是在用户和物品很多的时候。因此实践中该参数常作为训练效果与系统开销之间的调节参数。通常,其合理取值为10到200。
* iterations:对应运行时的迭代次数。ALS能确保每次迭代都能降低评级矩阵的重建误差,但一般经少数次迭代后ALS模型便已能收敛为一个比较合理的好模型。这样,大部分情况下都没必要迭代太多次(10次左右一般就挺好)。
* lambda:该参数控制模型的正则化过程,从而控制模型的过拟合情况。其值越高,正则化越严厉。该参数的赋值与实际数据的大小、特征和稀疏程度有关。和其他的机器学习模型一样,正则参数应该通过用非样本的测试数据进行交叉验证来调整。  

作为示例,这里将使用的rank、iterations和lambda参数的值分别为50、10和0.01:


In [15]:
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import Rating

raw_data = sc.textFile('hdfs://master:9000/movieLens/ml-100k/u.data')
#数据集u.data中四个字段分别表示用户ID、电影ID、评分、时间戳
print('raw data sample:', raw_data.map(lambda x: x.split('\t')).take(3))

raw_rating = raw_data.map(lambda x: x.split('\t')[:3])
ratings = raw_rating.map(lambda x: Rating(x[0], x[1], x[2]))
ratings.cache()
print('rating data sample:', ratings.take(3))

#训练推荐模型
als_model = ALS.train(ratings, 50, 5, 0.1)

raw data sample: [['196', '242', '3', '881250949'], ['186', '302', '3', '891717742'], ['22', '377', '1', '878887116']]
rating data sample: [Rating(user=196, product=242, rating=3.0), Rating(user=186, product=302, rating=3.0), Rating(user=22, product=377, rating=1.0)]


In [17]:
from pyspark.mllib.linalg import Vectors

movie_factors = als_model.productFeatures().map(lambda x: (x[0], Vectors.dense(x[1])))
movie_vectors = movie_factors.map(lambda x: x[1])

user_factors = als_model.userFeatures().map(lambda x: (x[0], Vectors.dense(x[1])))
user_vectors = user_factors.map(lambda x: x[1])

### 归一化
在训练聚类模型之前，有必要观察一下输入数据的相关因素特征向量的分布，这可以告诉我们是否需要对训练数据进行归一化  
从下面的结果来看，没有发现特别立群的点会影响聚类结果，因此也就没必要进行归一化

In [20]:
from pyspark.mllib.linalg.distributed import RowMatrix

movie_matrix = RowMatrix(movie_vectors)
user_matrix = RowMatrix(user_vectors)

from pyspark.mllib.stat import MultivariateStatisticalSummary
desc_movie_matrix = MultivariateStatisticalSummary(movie_matrix.rows)
desc_user_matrix = MultivariateStatisticalSummary(user_matrix.rows)

print('Movie factors mean:',desc_movie_matrix.mean())
print('Movie factors variance:',desc_user_matrix.mean())
print('User factors mean:',desc_movie_matrix.variance())
print('User factors variance:',desc_user_matrix.variance())

Exception ignored in: <bound method JavaModelWrapper.__del__ of <pyspark.mllib.stat._statistics.MultivariateStatisticalSummary object at 0x7fc32e8686d8>>
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/mllib/common.py", line 142, in __del__
    self._sc._gateway.detach(self._java_model)
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1870, in detach
AttributeError: 'RDD' object has no attribute '_detach'


Movie factors mean: [ -2.46042605e-01   4.09215370e-01   1.71205918e-01  -3.99813389e-02
   4.45088787e-01   2.57521659e-01  -3.37851754e-01   1.94622614e-01
  -8.46795980e-02   3.89513004e-02   1.82929406e-01   3.34531888e-01
  -1.26887030e-01  -1.76767938e-01   1.12459881e-01  -1.18506279e-01
   3.36070223e-02  -1.85314937e-01  -3.09632505e-01  -1.32173863e-01
   3.65567397e-04   9.90960281e-02   4.80696659e-02  -1.92687931e-01
   9.01135470e-02  -1.40105695e-01   1.69925708e-01   7.14769749e-02
  -2.02924173e-01   7.92776238e-02  -1.70273464e-01  -9.42535695e-02
  -3.57966176e-01   2.97810137e-01  -1.99743315e-01  -1.16254592e-01
   1.86592883e-01   2.46867168e-01  -1.36812178e-01   1.54292215e-01
   5.67643408e-02   3.49779028e-02   1.36715592e-01  -1.74479573e-01
   1.37425726e-01  -2.49777473e-01   2.44814970e-01   2.06469067e-01
   1.20410172e-01  -6.83830333e-02]
Movie factors variance: [-0.40489061  0.60710787  0.27739984 -0.00444145  0.62699867  0.37975445
 -0.48969531  0.273

## 训练聚类模型

In [27]:
from pyspark.mllib.clustering import KMeans
num_clusters = 5
num_iterations = 20
movie_cluster_model = KMeans.train(movie_vectors, num_clusters, 100)
user_cluster_model = KMeans.train(user_vectors, num_clusters, num_iterations)
predictions = movie_cluster_model.predict(movie_vectors)
print('前十个样本的预测标签：' + "," .join([str(i) for i in predictions.take(10)]))

前十个样本的预测标签：4,0,2,4,0,1,4,4,4,1
